In [1]:
%reload_ext autoreload
%autoreload 2
import csv
from CIOGame import *
import pandas as pd
import numpy as np
from sklearn import linear_model, preprocessing, ensemble
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score
from gurobipy import *

# Reports in previous semesters

In [ ]:
file = 'CMO_Controlling-Tool.xlsx'
# Load spreadsheet
xl = pd.ExcelFile(file)
df=xl.parse('Marketing KPIs').drop(list(range(10,18)))
df=df.set_index([list(df.iloc[:,0])]).iloc[:,1:].transpose().drop(['Prozessqualität'],axis=1)
df

In [ ]:
cio_game.correlation_matrix(df)

In [ ]:
df_corr=df.corr()
df_corr

In [ ]:
df_corr.abs().sort_values(by=['     Global'],ascending=False)#.drop(["Marktanteil (New Savings)","Kundenzufriedenheit"])

# CIO regression
## Read data sets from .xlsm

In [97]:
cio_game=CIOGame(5)
df_cmo=cio_game.read_from_exel().T
df_cmo

Created CIOGame Object for period 5


,Car Financing Loans,Customer Savings,Requests to be processed LowPriceCars,Requests to be processed MidPriceCars,Requests to be processed HighPriceCars,Contracts succeeded LowPriceCars,Contracts succeeded MidPriceCars,Contracts succeeded HighPriceCars,Service Transactions to be processed loans,Service Transactions succeeded loans,...,Service Transactions to be processed savings,Service Transactions succeeded savings,Customer Satisfaction,Marketing Efficiency for product Car Financing Loans,Marketing Efficiency for product Savings Account,Marketing Expenditures Global,Marketing Expenditures Product Loans,Marketing Expenditures Product Savings,Interest Rate Car Financing Loans,Interest Rate Customer Savings
0,1.806205e+09,1.084270e+09,59391.8,74239.8,14848.0,18460.7,23075.9,4615.2,216749.4,173290.4,...,162640.8,156517.6,0.8332,0.7295,0.6915,4500000.0,2220000.0,1200000.0,0.120,0.030
1,2.595910e+09,9.625200e+08,56685.5,70856.9,14171.4,39239.7,49049.6,9809.9,311509.6,298465.7,...,144377.4,135516.1,0.8492,0.6938,0.6576,4500000.0,2020000.0,1400000.0,0.120,0.035
2,3.387190e+09,1.138230e+09,62877.9,78597.4,15719.5,41754.9,52193.6,10438.7,406463.3,327376.3,...,170734.6,162806.6,0.8681,0.7884,0.7680,4500000.0,2220000.0,1200000.0,0.123,0.035
3,3.732180e+09,2.021880e+09,46496.4,58120.5,11624.1,35495.5,44369.4,8873.9,447862.1,425422.3,...,303281.3,285108.1,0.8929,0.9085,0.9026,3300000.0,1100000.0,900000.0,0.130,0.032
4,4.475400e+09,3.038280e+09,54842.7,68553.3,13710.7,50091.1,62613.9,12522.8,537047.4,524852.8,...,455742.2,428042.9,0.9269,0.9689,0.9684,3600000.0,1200000.0,1440000.0,0.137,0.030
5,5.574000e+09,5.433580e+09,63389.0,79236.3,15847.3,60304.8,75381.0,15076.2,668880.0,651918.1,...,815036.6,668233.4,0.9663,0.9854,0.9854,4500000.0,1500000.0,5000000.0,0.147,0.023


In [98]:
df_cmo.keys()

Index(['Car Financing Loans', 'Customer Savings',
       'Requests to be processed LowPriceCars',
       'Requests to be processed MidPriceCars',
       'Requests to be processed HighPriceCars',
       'Contracts succeeded LowPriceCars', 'Contracts succeeded MidPriceCars',
       'Contracts succeeded HighPriceCars',
       'Service Transactions to be processed loans',
       'Service Transactions succeeded loans',
       'Requests to be processed savings', 'Contracts succeeded savings',
       'Service Transactions to be processed savings',
       'Service Transactions succeeded savings', 'Customer Satisfaction',
       'Marketing Efficiency for product Car Financing Loans',
       'Marketing Efficiency for product Savings Account',
       'Marketing Expenditures Global', 'Marketing Expenditures Product Loans',
       'Marketing Expenditures Product Savings',
       'Interest Rate Car Financing Loans', 'Interest Rate Customer Savings'],
      dtype='object', name='')

In [99]:
sum_contracts_succeeded_loans=df_cmo[[
    'Contracts succeeded LowPriceCars',
    'Contracts succeeded MidPriceCars',
    'Contracts succeeded HighPriceCars'
]].sum(axis=1).rename("Contracts succeeded Total")

df_loans=df_cmo[[
    "Car Financing Loans",
    'Service Transactions succeeded loans',
]]
df_loans=df_loans.T.append(sum_contracts_succeeded_loans).T

## Find the best regression model

In [100]:
y=df_loans["Car Financing Loans"]
X=df_loans.drop(["Car Financing Loans"],axis=1)
regr_loans=cio_game.find_best_regr(X[:-1],y[:-1],X.iloc[len(X)-1],y.iloc[len(y)-1])

[0.0072008838816408627, 0.1981230339508222, 0.16317272810997352, 0.020450840665972025, 0.032614101431490683]
5574000000.0
5509205729.29
                               features  estimatedCoefficients
0  Service Transactions succeeded loans            6848.753335
1             Contracts succeeded Total            4187.009921


## Find the relationship of amount contracts in last period and amount of service transactions in this period

In [101]:
df_ta_loans=df_loans.drop(["Car Financing Loans"],axis=1).T.append(df_cmo["Service Transactions to be processed loans"].shift(-1).rename("Service Transactions to be processed loans next Period"))
df_ta_loans=df_ta_loans.append(df_cmo["Service Transactions succeeded loans"].shift(1).rename("Service Transactions succeeded loans last Period")).T
y=df_ta_loans["Service Transactions to be processed loans next Period"][1:]
X=df_ta_loans.drop(["Service Transactions to be processed loans next Period"],axis=1)[1:]
regr_ta_loans=cio_game.find_best_regr(X[:-1],y[:-1],X.iloc[len(X)-1],y.iloc[len(y)-1])

[0.1159725350443257, 0.059820206270812859, 0.028533388938561721, 0.058510362172537578]
nan
813679.61353
                                           features  estimatedCoefficients
0              Service Transactions succeeded loans               1.056644
1                         Contracts succeeded Total               0.093373
2  Service Transactions succeeded loans last Period               0.081993


In [104]:
Service_Transactions_to_be_processed_loans_next_Period=813679.61353
Service_Transactions_succeeded_loans_last_Period=651918.1
for new_loans in range(150000,700000,50000):
    X_test=pd.Series({
        "Service Transactions succeeded loans":Service_Transactions_to_be_processed_loans_next_Period,
             "Contracts succeeded Total":new_loans,
             "Service Transactions succeeded loans last Period":Service_Transactions_succeeded_loans_last_Period})
    loans_pre=regr_loans.predict([Service_Transactions_to_be_processed_loans_next_Period,new_loans])
    #if loans_pre>=8000000000:
    print("%g: %g"%(new_loans,loans_pre))

150000: 6.61388e+09
200000: 6.82323e+09
250000: 7.03258e+09
300000: 7.24193e+09
350000: 7.45128e+09
400000: 7.66063e+09
450000: 7.86998e+09
500000: 8.07933e+09
550000: 8.28868e+09
600000: 8.49803e+09
650000: 8.70738e+09


/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/anaconda/

## need 500000 loans TAs 

In [52]:
X=df_cmo[[
    'Marketing Efficiency for product Car Financing Loans',
    'Marketing Expenditures Global', 
    'Marketing Expenditures Product Loans',
    #'Interest Rate Car Financing Loans'
]]
y=df_cmo[[
    'Requests to be processed LowPriceCars',
    'Requests to be processed MidPriceCars',
    'Requests to be processed HighPriceCars'
]].sum(axis=1).rename("Requests to be processed Total")

In [53]:
regr_exp_loans=cio_game.find_best_regr(X[:-1],y[:-1],X.loc[5],y.loc[5])

[0.0032028474141847507, 0.011114016339803356, 0.0038689715197578467, 0.016259421258684958, 0.01033872553199866]
158472.6
176663.577262
                                            features  estimatedCoefficients
0  Marketing Efficiency for product Car Financing...          139892.062769
1                      Marketing Expenditures Global               0.037642
2               Marketing Expenditures Product Loans               0.011236


# Savings

In [58]:
df_cmo.keys()

Index(['Car Financing Loans', 'Customer Savings',
       'Requests to be processed LowPriceCars',
       'Requests to be processed MidPriceCars',
       'Requests to be processed HighPriceCars',
       'Contracts succeeded LowPriceCars', 'Contracts succeeded MidPriceCars',
       'Contracts succeeded HighPriceCars',
       'Service Transactions to be processed loans',
       'Service Transactions succeeded loans',
       'Requests to be processed savings', 'Contracts succeeded savings',
       'Service Transactions to be processed savings',
       'Service Transactions succeeded savings', 'Customer Satisfaction',
       'Marketing Efficiency for product Car Financing Loans',
       'Marketing Efficiency for product Savings Account',
       'Marketing Expenditures Global', 'Marketing Expenditures Product Loans',
       'Marketing Expenditures Product Savings',
       'Interest Rate Car Financing Loans', 'Interest Rate Customer Savings'],
      dtype='object', name='')

In [59]:
df_savings=df_cmo[[
    "Customer Savings",
    'Contracts succeeded savings',
    'Service Transactions succeeded savings'
]]
df_savings

,Customer Savings,Contracts succeeded savings,Service Transactions succeeded savings
0,1.084270e+09,136564.3,156517.6
1,9.625200e+08,172480.4,135516.1
2,1.138230e+09,308382.6,162806.6
3,2.021880e+09,772732.2,285108.1
4,3.038280e+09,1048853.7,428042.9
5,5.433580e+09,2071513.3,668233.4


In [106]:
y=df_savings["Customer Savings"]
X=df_savings.drop(["Customer Savings"],axis=1)
regr_savings=cio_game.find_best_regr(X[:-1],y[:-1],X.loc[5],y.loc[5])

[0.020318101782805575, 0.024067394866507996, 0.0099115995784717636, 0.0039962838174385507, 0.011232697424401361]
5433580000.0
4778602298.93
                                 features  estimatedCoefficients
0             Contracts succeeded savings              88.500701
1  Service Transactions succeeded savings            6874.250422


## Service is more importent

In [107]:
df_ta_savings=df_savings.drop(["Customer Savings"],axis=1).T.append(df_cmo["Service Transactions to be processed savings"].shift(-1).rename("Service Transactions to be processed savings next Period"))
df_ta_savings=df_ta_savings.append(df_cmo["Service Transactions succeeded savings"].shift(1).rename("Service Transactions succeeded savings last Period")).T
y=df_ta_savings["Service Transactions to be processed savings next Period"][1:]
X=df_ta_savings.drop(["Service Transactions to be processed savings next Period"],axis=1)[1:]
regr_ta_savings=cio_game.find_best_regr(X[:-1],y[:-1],X.iloc[len(X)-1],y.iloc[len(y)-1])

[0.87281607313856124, 0.34384509954505771, 0.48908642224792231, 0.75634653025656773]
nan
1466423.24041
                                            features  estimatedCoefficients
0                        Contracts succeeded savings               0.236888
1             Service Transactions succeeded savings               0.897785
2  Service Transactions succeeded savings last Pe...               1.353701


In [108]:
df_ta_savings

,Contracts succeeded savings,Service Transactions succeeded savings,Service Transactions to be processed savings next Period,Service Transactions succeeded savings last Period
0,136564.3,156517.6,144377.4,NaN
1,172480.4,135516.1,170734.6,156517.6
2,308382.6,162806.6,303281.3,135516.1
3,772732.2,285108.1,455742.2,162806.6
4,1048853.7,428042.9,815036.6,285108.1
5,2071513.3,668233.4,NaN,428042.9


In [114]:
Service_Transactions_to_be_processed_savings_next_Period=1466423.24041
for new_savings in range(1000000,4000000,500000):
    savings_pre=regr_savings.predict([Service_Transactions_to_be_processed_savings_next_Period,new_savings])
    if savings_pre>=8000000000:
        print("%g: %g"%(new_savings,savings_pre))

1.5e+06: 1.04428e+10
2e+06: 1.38799e+10
2.5e+06: 1.73171e+10
3e+06: 2.07542e+10
3.5e+06: 2.41913e+10


/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/anaconda/

In [119]:
cio_game.find_best_solution(regr_exp_loans,best_regr_savings,450000,500000)

Optimize a model with 2 rows, 3 columns and 4 nonzeros
Variable types: 0 continuous, 3 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e-02, 3e-01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+05, 5e+05]
Found heuristic solution: objective 4.16702e+07
Presolve removed 2 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds
Thread count was 1 (of 4 available processors)

Solution count 2: 1.19547e+07 4.16702e+07 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.195471700000e+07, best bound 1.195471700000e+07, gap 0.0000%
Global_expenditure: 1.19547e+07
Saving_expenditure: -0
Loan_expenditure: -0
Obj: 1.19547e+07


# test

In [115]:
global_expenditure=np.array([4500000,4500000,4500000,3300000,3600000,4500000])

savings_intrests_rate=np.array([0.0300,0.0350,0.0350,0.0320,0.03,0.023])
savings_expenditure=np.array([1200000,1400000,1200000,900000,1440000,5000000])
savings_marketing_efficiency=np.array([0.6915,0.6576,0.7680,0.9026,0.9684,0.9864])

loans_intrests_rate=np.array([0.12,0.12,0.123,0.13,0.137,0.147])
loans_expenditure=np.array([2220000,2020000,2220000,1100000,1200000,1500000])
loans_marketing_efficiency=np.array([0.7295,0.6938,0.7884,0.9085,0.9689,0.9854])

savings_requests=np.array([902795.10,951090.40,993601.10,837795.40,1114117.8,2376077.1 ])
loans_requests=np.array([148479.60,141713.80 ,157194.80 ,116241.00,137106.70,158472.6])
#loans_requests=np.array([74239.80 ,70856.90 ,78597.40 ,58120.50 ,68553.30 ,79236.3])

df_savings=pd.DataFrame({
    "global_expenditure":global_expenditure,
    "savings_expenditure":savings_expenditure,
    "savings_intrests_rate":savings_intrests_rate,
    "savings_marketing_efficiency":savings_marketing_efficiency,
    "savings_requests":savings_requests})[:-1]
df_loans=pd.DataFrame({
    "global_expenditure":global_expenditure,
    "loans_expenditure":loans_expenditure,
    "loans_intrests_rate":loans_intrests_rate,
    "loans_marketing_efficiency":loans_marketing_efficiency,
    "loans_requests":loans_requests})[:-1]

In [116]:
df_savings

,global_expenditure,savings_expenditure,savings_intrests_rate,savings_marketing_efficiency,savings_requests
0,4500000,1200000,0.030,0.6915,902795.1
1,4500000,1400000,0.035,0.6576,951090.4
2,4500000,1200000,0.035,0.7680,993601.1
3,3300000,900000,0.032,0.9026,837795.4
4,3600000,1440000,0.030,0.9684,1114117.8


In [117]:
df_loans["global_expenditure"]

0    4500000
1    4500000
2    4500000
3    3300000
4    3600000
Name: global_expenditure, dtype: int64

In [118]:
y_savings=df_savings["savings_requests"]
X_savings=df_savings.drop(["savings_requests"],axis=1)

df_savings_pre=pd.DataFrame({
    "global_expenditure":[4500000],
    "savings_expenditure":[5000000],
    "savings_intrests_rate":[0.02],
    "savings_marketing_efficiency":[0.9854]
})

best_regr_savings=cio_game.find_best_regr(X_savings,y_savings,df_savings_pre,2376077.1)

[0.020631599322028515, 0.038217451996219065, 0.034250256516569655, 0.13449534371837213, 0.075853455726701213]
2376077.1
2371377.45461
                       features  estimatedCoefficients
0            global_expenditure               0.158608
1           savings_expenditure               0.308537
2         savings_intrests_rate           -7289.507342
3  savings_marketing_efficiency          944004.454426


In [ ]:
y_loans=df_loans["loans_requests"]
X_loans=df_loans.drop(["loans_requests"],axis=1)

df_loans_pre=pd.DataFrame({
    "global_expenditure":[4500000],
    "savings_expenditure":[1500000],
    "savings_intrests_rate":[0.147],
    "savings_marketing_efficiency":[0.9854]
})

best_regr_loans=find_best_regr(X_loans,y_loans,df_loans_pre,158472.6)

In [ ]:
best_regr_savings.coef_

In [ ]:
best_regr_loans.coef_